In [1]:
import pandas as pd
import os
import numpy as np

import Utils as ut
import CortesAlignmentFile as ca
import mySampler as ms
 
from sklearn.model_selection import train_test_split, ShuffleSplit


/Users/danilo/.conda/envs/bcb/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [2]:
d_clinical = pd.read_csv(os.path.join('data', 'dataset_clinical_cleaned.csv'))
d_genetic = pd.read_csv(os.path.join('data', 'dataset_genetic_cleaned_noOHE.csv'))
d_vampire = pd.read_csv(os.path.join('data', 'dataset_vampire_cleaned.csv'))
outputs = pd.read_csv(os.path.join('data', 'outputs_cleaned.csv'))

## Heart Attack

In [3]:
y_class = outputs["cvd_fail"].values
meaningful_idxs = np.where(y_class==1)
y = outputs["cvd_time_age"].values[meaningful_idxs]
C = d_clinical.values[meaningful_idxs]
G = d_genetic.values[meaningful_idxs]
V = d_vampire.values[meaningful_idxs]

In [4]:
# COMPUTATIONAL COMPLEXITY: Reduce #samples
#tr_idx, ts_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.75).split(C, y))
C_, C_test, G_, G_test, V_, V_test, y_, y_test = train_test_split(C, G, V, y, test_size=0.25)

In [5]:
ds_list = [C_, G_, V_]
ds_test = [C_test, G_test, V_test]
ds_names = ['clinic', 'genetic', 'vampire']

## Basic approach

In [6]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3, 7], 'gaussian':[0.1, 0.5, 0.7]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, Ptype="regression")

In [7]:
result1 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True)


1 split out of 3 ...
	Working on config 1 of 2: {'linear': [0.5], 'polynomial': [2, 3, 7], 'gaussian': [0.1, 0.5, 0.7]}
Fold no. 1
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
Fold no. 2
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
Fold no. 3
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
Validation complete, config selected:[[0.5, 7, 0.7], [0.5, 2, 0.7], [0.5, 7, 0.5]]
	Result of 1:
CA: 0.005816683923558076
Average error: 7.6558238336786255
Error variance: 33.1905957788202
[linear:0.5, polynomial:7, gaussian:0.7, ]
[linear:0.5, polynomial:2, gaussian:0.7, ]
[linear:0.5, polynomial:7, gaussian:0.5, ]

eta vector: [ 5.05369904e-07 -2.96107615e-28  1.96311182e-01  5.24043314e-07
 -5.22618858e-05  5.88321875e-01  1.77848071e-16  7.67597879e-60
 -7.84435650e-01]


	Completed in 3.85 minutes
	Working on config 2 of 2: {'linear': [0.2], 'polynomial': 

In [8]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names, Ptype='regression')

NameError: name 'bestOverDict' is not defined

In [ ]:
w_dict

In [9]:
result1.performancesFeatures()

statistics of configuration 1
{'config': {'linear': [[0.5, 0.5, 0.5], [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]], 'polynomial': [[7, 2, 7], [7, 7, 2], [7, 2, 7]], 'gaussian': [[0.7, 0.7, 0.5], [0.5, 0.7, 0.5], [0.7, 0.7, 0.7]]}, 'CA': (0.005463499128406509, 8.544571472635737e-08), 'meanErr': (8.075205750114586, 0.19714172896473034), 'varErr': (34.59266423175841, 1.3424172993063515), 'eta': (array([ 1.90172446e-07, -1.10726031e-28,  9.83482858e-02,  1.95319144e-07,
       -1.88986298e-05, -2.90074621e-01, -6.88143677e-16,  2.83015634e-21,
        1.91802361e-01]), array([4.98367379e-14, 1.72286632e-56, 6.31746633e-03, 5.42223790e-14,
       5.59773507e-10, 3.86094461e-01, 1.32338063e-30, 1.60195698e-41,
       4.76984255e-01]))}
statistics of configuration 2
{'config': {'linear': [[0.2, 0.2, 0.2], [0.2, 0.2, 0.2], [0.2, 0.2, 0.2]], 'polynomial': [[8, 8, 4], [5, 8, 5], [8, 8, 5]], 'gaussian': [[1, 1, 1], [1, 1, 0.7], [1, 0.7, 0.7]]}, 'CA': (0.005479936569181553, 3.5644110302376846e-08), 'meanErr'

## Kernel normalization

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3], 'gaussian':[0.1, 0.3, 0.6]}#,
               #{'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}
              ]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, Ptype="regression", normalize_kernels = True)

In [ ]:
result3 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result3.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names, Ptype='regression')

In [ ]:
w_dict

In [ ]:
result3.performancesFeatures()

## Normalized data

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3, 7], 'gaussian':[0.1, 0.5, 0.7]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, Ptype="regression", normalizing = True)

In [ ]:
result4 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[0, 3, 8, 13, 14], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result4.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names, Ptype='regression')

In [ ]:
w_dict

In [ ]:
result4.performancesFeatures()

## Origin Data Centering

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3, 7], 'gaussian':[0.1, 0.5, 0.7]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, Ptype="regression", centering = True)

In [ ]:
result5 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result5.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names, Ptype='regression')

In [ ]:
w_dict

In [ ]:
result5.performancesFeatures()

## Origin Data  Centering and Normalization

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3, 7], 'gaussian':[0.1, 0.5, 0.7]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, Ptype="regression", centering = True, normalizing = True)

In [ ]:
result6 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result6.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names, Ptype='regression')

In [ ]:
w_dict

In [ ]:
result6.performancesFeatures()

## Normalization, Kernel Normalization

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3,4], 'gaussian':[0.1, 0.3, 0.6]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}
              ]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, Ptype="regression", normalizing = True, normalize_kernels = True)

In [ ]:
result10 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result10.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names, Ptype='regression')

In [ ]:
w_dict

In [ ]:
result10.performancesFeatures()

## Centering, Kernel Normalization

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3,4], 'gaussian':[0.1, 0.3, 0.6]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}
              ]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, Ptype="regression", centering = True, normalize_kernels = True)

In [ ]:
result11 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result11.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names, Ptype='regression')

In [ ]:
w_dict

In [ ]:
result11.performancesFeatures()

## Centering, Normalization, Kernel Normalization

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3,4], 'gaussian':[0.1, 0.3, 0.6]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}
              ]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, Ptype="regression", centering = True, normalizing = True, normalize_kernels = True)

In [ ]:
result7 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result7.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names, Ptype='regression')

In [ ]:
w_dict

In [ ]:
result7.performancesFeatures()

## L2 Penalty, Centering, K-Normalization

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3,4], 'gaussian':[0.1, 0.3, 0.6]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}
              ]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, Ptype="regression", lamb = 0.5, centering = True, normalize_kernels = False)

In [ ]:
result9 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result9.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names, Ptype='regression')

In [ ]:
w_dict

In [ ]:
result9.performancesFeatures()

## Sparsity in eta, Centering, K-Normalization

In [3]:
y_class = outputs["cvd_fail"].values
meaningful_idxs = np.where(y_class==1)
y = outputs["cvd_time_age"].values[meaningful_idxs]
C = d_clinical.values[meaningful_idxs]
G = d_genetic.values[meaningful_idxs]
V = d_vampire.values[meaningful_idxs]

In [4]:
tr_idx, ts_idx = next(ShuffleSplit(n_splits=1, test_size=0.5).split(C, y))

In [5]:
y_ = y[tr_idx]
y_test = y[ts_idx]
C_ = C[tr_idx]
C_test = C[ts_idx]
G_ = G[tr_idx]
G_test = G[ts_idx]
V_ = V[tr_idx]
V_test = V[ts_idx]

In [6]:
ds_list = [C_, G_, V_]
ds_test = [C_test, G_test, V_test]
ds_names = ['clinic', 'genetic', 'vampire']

In [7]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3], 'gaussian':[0.1, 0.3]}#,
               #{'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}
              ]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, Ptype="regression", sparsity = 0.3, centering = True, normalize_kernels = True)

In [9]:
result8 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])


1 split out of 3 ...
	Working on config 1 of 1: {'linear': [0.5], 'polynomial': [2, 3], 'gaussian': [0.1, 0.3]}
Fold no. 1
Fold no. 2
Fold no. 3
Validation complete, config selected:[[0.5, 3, 0.3], [0.5, 2, 0.1], [0.5, 2, 0.1]]
	Result of 1:
CA: 0.006847414199957387
Average error: 9.231676732599034
Error variance: 130.08956487754818
[linear:0.5, polynomial:3, gaussian:0.3, ]
[linear:0.5, polynomial:2, gaussian:0.1, ]
[linear:0.5, polynomial:2, gaussian:0.1, ]

eta vector: [ 0.05948608 -0.72685468  0.07174706  0.          0.          0.
  0.          0.68043812  0.        ]


	Completed in 2.3833333333333333 minutes

2 split out of 3 ...
	Working on config 1 of 1: {'linear': [0.5], 'polynomial': [2, 3], 'gaussian': [0.1, 0.3]}
Fold no. 1
Fold no. 2
Fold no. 3
Validation complete, config selected:[[0.5, 3, 0.1], [0.5, 2, 0.1], [0.5, 2, 0.1]]
	Result of 2:
CA: 0.006655651159134598
Average error: 119.55665577352771
Error variance: 921789.4865830898
[linear:0.5, polynomial:3, gaussian:0.1,

In [10]:
w_dict, w_list = result8.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names, Ptype='regression')

Perfomances computed for 1 dictionary settings:
	Average error: 168207463991250.94
	Error variance: 1.0854900425689533e+30


In [11]:
w_dict

[{'clinic': {'linear': 0.5, 'polynomial': 3, 'gaussian': 0.3},
  'genetic': {'linear': 0.5, 'polynomial': 2, 'gaussian': 0.1},
  'vampire': {'linear': 0.5, 'polynomial': 2, 'gaussian': 0.1}}]

In [12]:
result8.performancesFeatures()

statistics of configuration 1
{'config': {'linear': [[0.5, 0.5, 0.5], [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]], 'polynomial': [[3, 2, 2], [3, 2, 2], [2, 2, 2]], 'gaussian': [[0.3, 0.1, 0.1], [0.1, 0.1, 0.1], [0.3, 0.1, 0.3]]}, 'CA': (0.006792874679182674, 9.547165838852482e-09), 'meanErr': (51.733672541227996, 2349.176368785072), 'varErr': (313223.88149011304, 185227803540.31204), 'eta': (array([ 0.05005628, -0.86021719,  0.18915221,  0.        ,  0.        ,
        0.        ,  0.        ,  0.3460714 ,  0.        ]), array([0.00010462, 0.00952286, 0.03015834, 0.        , 0.        ,
       0.        , 0.        , 0.06219096, 0.        ]))}
